In [16]:
import logging
logging.basicConfig()
logger = logging.getLogger("Sklearn SVM")
logger.setLevel(logging.INFO)

from pyvi import ViTokenizer
from sklearn.preprocessing import MaxAbsScaler, MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from utils.data_loader import load_phoatis


class SVMClassifier:
    """ 
    * Support Vector Machine with linear kernel and 
        TF-IDF feturizer by default.
    * This class uses the Sklearn's Pipeline.
    * It also handles for multilabel problem by using
        MultiOutputClassifier from Sklearn.
    """
    def __init__(self,
                 kernel="linear",
                 vectorizer=TfidfVectorizer,
                 tokenizer=True,
                 scaler="maxabsscaler",
                 probability=True,
                 random_state=42,
                 num_cpu=-1):
        self.kernel = kernel
        self.random_state = random_state
        self.probability = probability
        self.num_cpu = num_cpu
        self.tokenizer = self.__tokenize if tokenizer else None

        # pipeline configuration
        self.vectorizer = vectorizer(token_pattern=r"(?u)\b\w+\b",
                                          tokenizer=self.tokenizer)
        if scaler == "maxabsscaler":
            self.scaler = MaxAbsScaler()

        self.classifier = SVC(kernel=self.kernel,
                              random_state=self.random_state,
                              verbose=1,
                              probability=self.probability)

        self.pipeline = Pipeline([
            ("vectorizer", self.vectorizer),
            ("scaler", self.scaler),
            ("clf", MultiOutputClassifier(self.classifier, n_jobs=self.num_cpu))
        ])
        
        # This gird has to depend on training set size
        # due to training time cost.
        self.grid_params = {
            "vectorizer__ngram_range": [(1, 1), (1, 2)],
            "vectorizer__max_df": [0.25, 0.5, 1],
            "clf__estimator__C": [1e-3, 1e-2, 0.1, 1]
        }

    def train(self, X, y, **kwargs):
        try:
            if kwargs["label_binarizer"]:
                self.label_binarizer = kwargs["label_binarizer"]
            else:
                self.label_binarizer = None

            self.tuner = GridSearchCV(estimator=self.pipeline,
                                      param_grid=self.grid_params,
                                      scoring="f1_macro",
                                      n_jobs=self.num_cpu,
                                      cv=5,
                                      verbose=3,
                                      error_score=0.,
                                      refit=True,
                                      return_train_score=True)
            # Search hyperparameter
            self.tuner.fit(X, y)

            # return best model
            logger.info(f"GridSearchCV Reports:\n{self.tuner.cv_results_}\n\n")
            logger.info(f"Best estimator: {self.tuner.best_estimator_}\n\n")
            logger.info(f"Best score: {self.tuner.best_score_}\n\n")
        except Exception as e:
            logger.error(f"{e}")
            return None
        
        self.model = self.tuner.best_estimator_
        return self.model
    
    def predict(self, X):
        predicted_result = self.model.predict(X)
        return predicted_result

    def evaluate(self, X, y):
        y_preds = self.model.predict(X)
        y_true = y

        # Sklearn's classification report
        clf_report = classification_report(y_true, y_pred,
                                           target_names=self.label_binarizer.classes_)
        logger.info(f"Classification Report:\n{clf_report}\n\n")

    def __tokenize(self, text):
        return ViTokenizer.tokenize(text).split()

In [2]:
""" How to use? """

# model = SVMClassifier()
# model.train(X, y)
# model.predict(X)

' How to use? '

## PhoATIS

In [3]:
""" Load & Preprocess Data """

train, val, test = load_phoatis("../data/phoatis/")
train_X, train_y = train[0], train[1]
val_X, val_y = val[0], val[1]
test_X, test_y = test[0], test[1]


def _preprocess(data):
    data = [x.split("\n")[0] for x in data]
    return data


# this remove \n
iters = list(map(_preprocess,
                [train_X, train_y,
                 val_X, val_y,
                 test_X, test_y]))
train_X, train_y = iters[0], iters[1]
val_X, val_y = iters[2], iters[3]
test_X, test_y = iters[4], iters[5]

In [4]:
def _transform_label(labels):
    labels = [label.split("#") for label in labels]
    return labels

train_y = _transform_label(train_y)
val_y = _transform_label(val_y)
test_y = _transform_label(test_y)

In [5]:
intent_label = []
with open("../data/phoatis/intent_label.txt", "r") as f:
    intent_label = f.readlines()
    intent_label = [label.split("\n")[0] for label in intent_label] # remove \n
    intent_label = intent_label[1:] # remove UNK label
    intent_label = _transform_label(intent_label)

print("Intents:\n\n", intent_label)
print(f"\nNumber of intents: {len(intent_label)}")

Intents:

 [['abbreviation'], ['aircraft'], ['aircraft', 'flight', 'flight_no'], ['airfare'], ['airfare', 'flight'], ['airline'], ['airline', 'flight_no'], ['airport'], ['capacity'], ['city'], ['city', 'flight_time'], ['distance'], ['flight'], ['flight', 'flight_no'], ['flight', 'flight_time'], ['flight_no'], ['flight_no', 'flight_time'], ['flight_time'], ['ground_fare'], ['ground_fare', 'ground_service'], ['ground_service'], ['meal'], ['quantity'], ['restriction']]

Number of intents: 24


In [6]:
lb = MultiLabelBinarizer()
train_y = lb.fit_transform(train_y)
val_y = lb.transform(val_y)
test_y = lb.transform(test_y)

/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['day_name'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


In [ ]:
model = SVMClassifier(tokenizer=False)
model = model.train(train_X, train_y, label_binarizer=lb)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
*
optimization finished, #iter = 22
obj = -0.019992, rho = 1.000058
nSV = 29, nBSV = 13
Total nSV = 29
*
optimization finished, #iter = 28
obj = -0.027986, rho = 1.000153
nSV = 39, nBSV = 19
Total nSV = 39
*
optimization finished, #iter = 25
obj = -0.017981, rho = 1.000123
nSV = 31, nBSV = 10
Total nSV = 31
*
optimization finished, #iter = 22
obj = -0.019991, rho = 1.000188
nSV = 32, nBSV = 12
Total nSV = 32
*
optimization finished, #iter = 25
obj = -0.019986, rho = 1.000214
nSV = 33, nBSV = 11
Total nSV = 33
*
optimization finished, #iter = 31
obj = -0.025968, rho = 0.999991
nSV = 42, nBSV = 15
Total nSV = 42
*
optimization finished, #iter = 28
obj = -0.019977, rho = 1.000210
nSV = 34, nBSV = 10
Total nSV = 34
*
optimization finished, #iter = 21
obj = -0.021989, rho = 1.000310
nSV = 29, nBSV = 16
Total nSV = 29
*
optimization finished, #iter = 23
obj = -0.019979, rho = 0.999922
nSV = 33, nBSV = 13
Total nSV = 33
*
optimizat


optimization finished, #iter = 10
obj = -0.009994, rho = 1.000820
nSV = 15, nBSV = 7
Total nSV = 15
*
optimization finished, #iter = 287
obj = -0.304829, rho = 0.999414
nSV = 353, nBSV = 278
Total nSV = 353

optimization finished, #iter = 23
obj = -0.021991, rho = 1.000119
nSV = 34, nBSV = 13
Total nSV = 34
*
optimization finished, #iter = 20
obj = -0.009970, rho = 1.001702
nSV = 22, nBSV = 5
Total nSV = 22
*
optimization finished, #iter = 21
obj = -0.021980, rho = 0.999848
nSV = 30, nBSV = 15
Total nSV = 30
*
optimization finished, #iter = 44
obj = -0.021961, rho = 1.000673
nSV = 51, nBSV = 12
Total nSV = 51
*
optimization finished, #iter = 27
obj = -0.025967, rho = -1.000839
nSV = 39, nBSV = 19
Total nSV = 39
**
optimization finished, #iter = 9
obj = -0.009993, rho = 1.001063
nSV = 13, nBSV = 7
Total nSV = 13
*
optimization finished, #iter = 21
obj = -0.025988, rho = 1.001300
nSV = 32, nBSV = 21
Total nSV = 32
*
optimization finished, #iter = 51
obj = -0.043945, rho = 1.001368
nSV =

**
optimization finished, #iter = 63
obj = -0.019959, rho = 0.999798
nSV = 70, nBSV = 10
Total nSV = 70

optimization finished, #iter = 23
obj = -0.015982, rho = 1.000054
nSV = 28, nBSV = 9
Total nSV = 28
**
optimization finished, #iter = 32

optimization finished, #iter = 22
obj = -0.017964, rho = 1.000699
nSV = 36, nBSV = 11
Total nSV = 36
obj = -0.025967, rho = 1.000144
nSV = 34, nBSV = 19
Total nSV = 34
*
optimization finished, #iter = 17
obj = -0.021985, rho = 1.001363
nSV = 27, nBSV = 18
Total nSV = 27

optimization finished, #iter = 67
obj = -0.053944, rho = 1.000915
nSV = 79, nBSV = 38
Total nSV = 79
*
optimization finished, #iter = 11
obj = -0.013990, rho = -1.001407
nSV = 17, nBSV = 11
Total nSV = 17
*
optimization finished, #iter = 22
obj = -0.019989, rho = 1.000511
nSV = 28, nBSV = 15
Total nSV = 28
***
optimization finished, #iter = 49
obj = -0.019948, rho = 0.999895
nSV = 58, nBSV = 11
Total nSV = 58
*
optimization finished, #iter = 170
obj = -0.166439, rho = 0.998995
nSV

*
optimization finished, #iter = 20
obj = -0.005985, rho = 1.000587
nSV = 20, nBSV = 3
Total nSV = 20
*
optimization finished, #iter = 26
obj = -0.027990, rho = -1.000093
nSV = 40, nBSV = 22
Total nSV = 40
*
optimization finished, #iter = 117
obj = -0.107375, rho = 1.003113
nSV = 128, nBSV = 88
Total nSV = 128
*
optimization finished, #iter = 25
obj = -0.011965, rho = 1.001558
nSV = 29, nBSV = 6
Total nSV = 29
*
optimization finished, #iter = 26
obj = -0.019980, rho = 1.000972
nSV = 32, nBSV = 11
Total nSV = 32
*
optimization finished, #iter = 23
obj = -0.025985, rho = -1.000476
nSV = 35, nBSV = 19
Total nSV = 35
*
optimization finished, #iter = 74
obj = -0.027936, rho = 0.999799
nSV = 82, nBSV = 16
Total nSV = 82
*
optimization finished, #iter = 61
obj = -0.031772, rho = 0.999356
nSV = 76, nBSV = 19
Total nSV = 76
*
optimization finished, #iter = 58
obj = -0.023849, rho = 0.999434
nSV = 69, nBSV = 12
Total nSV = 69
*
optimization finished, #iter = 10
obj = -0.003993, rho = 1.000547
nS

*
optimization finished, #iter = 28
obj = -0.013957, rho = -1.002322
nSV = 34, nBSV = 8
*
optimization finished, #iter = 95
obj = -0.087627, rho = 1.002316
nSV = 101, nBSV = 76
Total nSV = 101
*
optimization finished, #iter = 46
obj = -0.017938, rho = 1.000034
nSV = 52, nBSV = 10
Total nSV = 52
*
optimization finished, #iter = 20
obj = -0.021966, rho = 0.999873
nSV = 31, nBSV = 14
*
optimization finished, #iter = 31
obj = -0.019986, rho = 1.000196
nSV = 37, nBSV = 11
Total nSV = 37
*
optimization finished, #iter = 12
obj = -0.013988, rho = -1.001631
nSV = 18, nBSV = 10
Total nSV = 18
*Total nSV = 34
Total nSV = 31
*
optimization finished, #iter = 23
obj = -0.017979, rho = 0.999818
nSV = 31, nBSV = 10
Total nSV = 31
*
optimization finished, #iter = 36
obj = -0.017970, rho = 1.000985
nSV = 40, nBSV = 10
Total nSV = 40
*
optimization finished, #iter = 24
obj = -0.023991, rho = 1.000129
nSV = 35, nBSV = 17
Total nSV = 35
*
optimization finished, #iter = 159
obj = -0.158392, rho = 0.998129


...

*
optimization finished, #iter = 381
obj = -2.307090, rho = 1.057821
nSV = 186, nBSV = 12
Total nSV = 186
*
optimization finished, #iter = 431
obj = -4.515005, rho = 0.888200
nSV = 247, nBSV = 34
Total nSV = 247
*
optimization finished, #iter = 295
obj = -1.760690, rho = 1.078110
nSV = 157, nBSV = 14
Total nSV = 157
*
optimization finished, #iter = 318
obj = -2.663402, rho = 0.990674
nSV = 175, nBSV = 18
Total nSV = 175
**
optimization finished, #iter = 491
obj = -5.045609, rho = 1.295442
nSV = 209, nBSV = 43
Total nSV = 209
.*
optimization finished, #iter = 2394
obj = -35.919593, rho = 0.597703
nSV = 999, nBSV = 335
Total nSV = 999
*
optimization finished, #iter = 425
obj = -2.446336, rho = -1.110318
*nSV = 185, nBSV = 12
Total nSV = 185
.*
optimization finished, #iter = 1177
obj = -58.283087, rho = -0.635279
nSV = 885, nBSV = 689
Total nSV = 885
*
optimization finished, #iter = 337
obj = -2.291976, rho = 1.108354
nSV = 168, nBSV = 16
Total nSV = 168
.*
optimization finished, #iter = 

*
optimization finished, #iter = 414
obj = -2.848717, rho = -1.065471
nSV = 181, nBSV = 15
Total nSV = 181
*
optimization finished, #iter = 514
obj = -1.450168, rho = -0.969654
nSV = 299, nBSV = 11
Total nSV = 299
*
optimization finished, #iter = 512
obj = -3.817333, rho = 1.092206
nSV = 217, nBSV = 30
Total nSV = 217
*
optimization finished, #iter = 158
obj = -0.973712, rho = 0.998171
nSV = 103, nBSV = 7
Total nSV = 103
*
optimization finished, #iter = 552
obj = -4.320639, rho = 0.945841
nSV = 283, nBSV = 31
Total nSV = 283
*
optimization finished, #iter = 385
obj = -2.702095, rho = 1.068792
nSV = 185, nBSV = 16
Total nSV = 185
*
optimization finished, #iter = 254
obj = -1.986658, rho = 0.994136
nSV = 144, nBSV = 16
Total nSV = 144
*
optimization finished, #iter = 63
obj = -0.711049, rho = 1.190696
nSV = 36, nBSV = 5
Total nSV = 36
*
optimization finished, #iter = 444
obj = -2.678374, rho = 1.062199
nSV = 196, nBSV = 18
Total nSV = 196
*
optimization finished, #iter = 458
obj = -4.598

*.*
optimization finished, #iter = 1078
obj = -56.761894, rho = -0.624632
nSV = 862, nBSV = 673
Total nSV = 862
*
optimization finished, #iter = 210
obj = -1.239093, rho = -0.986147
nSV = 133, nBSV = 9
Total nSV = 133
*
optimization finished, #iter = 817
obj = -11.486863, rho = 1.338553
nSV = 345, nBSV = 101
Total nSV = 345
*
optimization finished, #iter = 787
obj = -11.840070, rho = 1.210634
nSV = 342, nBSV = 106
Total nSV = 342
**
optimization finished, #iter = 490
obj = -1.391826, rho = -0.972844
nSV = 310, nBSV = 12
Total nSV = 310
*.*
optimization finished, #iter = 1096
obj = -55.874619, rho = -0.612081
nSV = 851, nBSV = 659
Total nSV = 851

optimization finished, #iter = 481
obj = -4.479004, rho = 0.955742
nSV = 257, nBSV = 30
Total nSV = 257
*
optimization finished, #iter = 523
obj = -5.215216, rho = -0.877213
nSV = 288, nBSV = 44
Total nSV = 288
*
optimization finished, #iter = 685
obj = -4.451575, rho = -1.130121
nSV = 240, nBSV = 35
Total nSV = 240
*
optimization finished, #i

obj = -32.150000, rho = 1.000000
nSV = 322, nBSV = 322
Total nSV = 322
*
optimization finished, #iter = 146
obj = -29.140000, rho = 1.000000
nSV = 292, nBSV = 292
Total nSV = 292
**
optimization finished, #iter = 278
*
optimization finished, #iter = 76
obj = -8.749999, rho = -0.999533
nSV = 92, nBSV = 87
Total nSV = 92
obj = -55.455000, rho = 1.000000
nSV = 556, nBSV = 556
Total nSV = 556
*
optimization finished, #iter = 967

optimization finished, #iter = 247
obj = -146.514992, rho = -0.999282
nSV = 1501, nBSV = 1469
Total nSV = 1501
*
optimization finished, #iter = 79
obj = -0.853060, rho = 1.199273
nSV = 45, nBSV = 6
Total nSV = 45
obj = -49.260000, rho = 1.000000
nSV = 494, nBSV = 494
Total nSV = 494
*
optimization finished, #iter = 143
obj = -28.535000, rho = 1.000000
nSV = 286, nBSV = 286
Total nSV = 286
*
optimization finished, #iter = 906
obj = -144.929994, rho = -0.999326
nSV = 1479, nBSV = 1453
Total nSV = 1479
**
optimization finished, #iter = 138
obj = -27.540000, rho = 1.0

*
optimization finished, #iter = 10
obj = -1.985000, rho = 1.000000
nSV = 20, nBSV = 20
Total nSV = 20
*
optimization finished, #iter = 225
obj = -1.739118, rho = 0.983016
nSV = 141, nBSV = 13
Total nSV = 141
*
optimization finished, #iter = 14
obj = -2.775000, rho = 1.000000
nSV = 28, nBSV = 28
Total nSV = 28
*
optimization finished, #iter = 10
obj = -1.975000, rho = 1.000000
nSV = 20, nBSV = 20
Total nSV = 20
*
optimization finished, #iter = 11
obj = -2.175000, rho = 1.000000
nSV = 22, nBSV = 22
Total nSV = 22
*
optimization finished, #iter = 615
*
optimization finished, #iter = 186
obj = -1.051163, rho = 0.988837
nSV = 122, nBSV = 9
Total nSV = 122
*
optimization finished, #iter = 11
obj = -2.190000, rho = 1.000000
nSV = 22, nBSV = 22
Total nSV = 22
obj = -5.106292, rho = -1.081926
nSV = 239, nBSV = 43
Total nSV = 239
*
optimization finished, #iter = 22
obj = -2.775000, rho = -0.999833
nSV = 30, nBSV = 27
Total nSV = 30
*
optimization finished, #iter = 275
*
optimization finished, #

## EMANDAI